In [1]:
import pandas as pd
import numpy as np
import requests  
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import string
import time
import matplotlib.pyplot as plt
ua = UserAgent()

In [2]:
df = pd.read_csv('dotahero.csv', header = None)
df.columns = ['Hero','Role','Attribute']
for col in df.columns:
    df[col] = df[col].astype(str).str.replace("'","")
df['Role'] = df['Role'].str.replace(" ","")
df['Attribute'] = df['Attribute'].str.replace(" ","")
df

,Hero,Role,Attribute
0,Abaddon,Support,STR
1,Alchemist,Core,STR
2,Ancient Apparition,Support,INT
3,AntiMage,Core,AGI
4,Arc Warden,Core,AGI
...,...,...,...
115,Windranger,Core,INT
116,Winter Wyvern,Support,INT
117,Witch Doctor,Support,INT
118,Wraith King,Core,STR


In [3]:
headers = {'User-Agent':ua.chrome}
page = requests.get("https://www.dotabuff.com/heroes/anti-mage/counters",headers = headers) 
soup = BeautifulSoup(page.content, 'html5lib')

In [4]:
def cleanstrip(string):
    return string.replace(' ','').replace('-','').replace("'",'').lower()
df['key'] = df['Hero'].apply(lambda x:cleanstrip(x))
df

,Hero,Role,Attribute,key
0,Abaddon,Support,STR,abaddon
1,Alchemist,Core,STR,alchemist
2,Ancient Apparition,Support,INT,ancientapparition
3,AntiMage,Core,AGI,antimage
4,Arc Warden,Core,AGI,arcwarden
...,...,...,...,...
115,Windranger,Core,INT,windranger
116,Winter Wyvern,Support,INT,winterwyvern
117,Witch Doctor,Support,INT,witchdoctor
118,Wraith King,Core,STR,wraithking


In [5]:
def scan_str(tag, string):
    s = string.find(tag)+len(tag)+2
    for i in range(s+1,len(string)):
        if(string[i]=="\"" or string[i]==' ' or string[i]=='"'):
            e = i
    if('jpg' in string[s:e]):
        e = string.find('jpg')+3
    return string[s:e]

In [6]:
ls = []
for i in soup.find_all('tr'):
    if(i.text.count('%')==2 and ('width: 20px' not in str(i))):
        mystr = i.text.split('%')[0]
        num = mystr[-4:]
        hero = mystr[:-4]
        for c in string.punctuation:
            if(c!='-' and c!='.'):
                num = num.replace(c,'')
                hero = hero.replace(c,'')
                
        if('-' in mystr):
            num = float(num.replace('-',''))*-1
        hero = hero.replace('-','').replace('1','')
        #print(hero, num)
        ls.append((cleanstrip(hero),num))
df2 = pd.DataFrame(ls)
df2.columns = ['key','Advantage']
df2 = df2.sort_values('key')#.drop_duplicates('key').reset_index(drop = True)
df2


,key,Advantage
93,abaddon,-1.73
56,alchemist,0.49
113,ancientapparition,-3.7
105,arcwarden,-2.69
10,axe,2.63
...,...,...
76,windranger,-0.42
103,winterwyvern,-2.56
67,witchdoctor,0.12
110,wraithking,-3.09


In [7]:
page = requests.get("https://www.dota2.com/heroes/",headers = headers) 
soup2 = BeautifulSoup(page.content, 'html5lib')

In [8]:
ls = []
for i in soup2.find_all('img'):
    i = str(i)
    if('heroHoverLarge' in i):
        #print(i)
        hero = scan_str('id',i)
        ind = hero.find('"')
        hero = hero[6:ind].replace('_','')
        
        link = scan_str('src',i)
        ind = link.find('"')
        link = link[:ind]
        
        ls.append((hero,link))
df1 = pd.DataFrame(ls)
df1.columns = ['key','image']
df1 = df1.replace('abyssalunderlord','underlord')
df1 = df1.replace('centaur','centaurwarrunner')
df1 = df1.replace('doombringer','doom')
df1 = df1.replace('furion','naturesprophet')
df1 = df1.replace('magnataur','magnus')
df1 = df1.replace('necrolyte','necrophos')
df1 = df1.replace('nevermore','shadowfiend')
df1 = df1.replace('obsidiandestroyer','outworlddestroyer')
df1 = df1.replace('rattletrap','clockwerk')
df1 = df1.replace('shredder','timbersaw')
df1 = df1.replace('skeletonking','wraithking')
df1 = df1.replace('treant','treantprotector')
df1 = df1.replace('windrunner','windranger')
df1 = df1.replace('wisp','io')
df1 = df1.replace('zuus','zeus')
df1 = df1.sort_values('key')

df1[:60]

,key,image
80,abaddon,https://cdn.cloudflare.steamstatic.com/apps/do...
9,alchemist,https://cdn.cloudflare.steamstatic.com/apps/do...
113,ancientapparition,https://cdn.cloudflare.steamstatic.com/apps/do...
23,antimage,https://cdn.cloudflare.steamstatic.com/apps/do...
98,arcwarden,https://cdn.cloudflare.steamstatic.com/apps/do...
66,axe,https://cdn.cloudflare.steamstatic.com/apps/do...
99,bane,https://cdn.cloudflare.steamstatic.com/apps/do...
112,batrider,https://cdn.cloudflare.steamstatic.com/apps/do...
4,beastmaster,https://cdn.cloudflare.steamstatic.com/apps/do...
82,bloodseeker,https://cdn.cloudflare.steamstatic.com/apps/do...


In [9]:
tdf = df.merge(df2, how = 'left',left_on = 'key',right_on = 'key')
tdf = tdf.merge(df1, how = 'left',left_on = 'key',right_on = 'key')
tdf = tdf[tdf.key!='antimage']
tdf = tdf.drop_duplicates().reset_index(drop = True)
tdf = tdf.drop('key',axis = 1)
tdf['Advantage'] = tdf['Advantage'].astype(float)
tdf.to_json('hero.json')
display(tdf[:60])
display(tdf[60:120])

,Hero,Role,Attribute,Advantage,image
0,Abaddon,Support,STR,-1.73,https://cdn.cloudflare.steamstatic.com/apps/do...
1,Alchemist,Core,STR,0.49,https://cdn.cloudflare.steamstatic.com/apps/do...
2,Ancient Apparition,Support,INT,-3.70,https://cdn.cloudflare.steamstatic.com/apps/do...
3,Arc Warden,Core,AGI,-2.69,https://cdn.cloudflare.steamstatic.com/apps/do...
4,Axe,Core,STR,2.63,https://cdn.cloudflare.steamstatic.com/apps/do...
5,Bane,Support,INT,0.45,https://cdn.cloudflare.steamstatic.com/apps/do...
6,Batrider,Core,INT,0.24,https://cdn.cloudflare.steamstatic.com/apps/do...
7,Beastmaster,Core,STR,1.38,https://cdn.cloudflare.steamstatic.com/apps/do...
8,Bloodseeker,Core,AGI,1.27,https://cdn.cloudflare.steamstatic.com/apps/do...
9,Bounty Hunter,Support,AGI,0.16,https://cdn.cloudflare.steamstatic.com/apps/do...


,Hero,Role,Attribute,Advantage,image
60,Naga Siren,Core,AGI,0.59,https://cdn.cloudflare.steamstatic.com/apps/do...
61,Natures Prophet,Core,INT,0.94,https://cdn.cloudflare.steamstatic.com/apps/do...
62,Necrophos,Core,INT,-2.91,https://cdn.cloudflare.steamstatic.com/apps/do...
63,Night Stalker,Core,STR,0.81,https://cdn.cloudflare.steamstatic.com/apps/do...
64,Nyx Assassin,Core,AGI,2.07,https://cdn.cloudflare.steamstatic.com/apps/do...
65,Ogre Magi,Support,INT,-2.10,https://cdn.cloudflare.steamstatic.com/apps/do...
66,Omniknight,Core,STR,-1.55,https://cdn.cloudflare.steamstatic.com/apps/do...
67,Oracle,Support,INT,-0.75,https://cdn.cloudflare.steamstatic.com/apps/do...
68,Outworld Destroyer,Core,INT,2.04,https://cdn.cloudflare.steamstatic.com/apps/do...
69,Pangolier,Core,AGI,0.70,https://cdn.cloudflare.steamstatic.com/apps/do...
